#### San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [69]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [70]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [71]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data = pd.read_csv('sfo_neighborhoods_census_data.csv', index_col="year")
neighborhood_data = pd.read_csv('neighborhoods_coordinates.csv')
sfo_data.head()
neighborhood_data.head()
housing_units_mean = sfo_data.groupby('year')['housing_units'].mean()
housing_units_mean_std = housing_units_mean.std()
ymin = housing_units_mean.min() - housing_units_mean_std
ymax = housing_units_mean.max() + housing_units_mean_std
average_price_rent = sfo_data.groupby('year')[['sale_price_sqr_foot', 'gross_rent']].mean()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [77]:

# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    return housing_units_mean.hvplot.bar(
        title='Housing Unit in San francisco from 2010 to 2016',
        ylim=[ymin,ymax],
        xlabel='Year',
        ylabel='Housing Units')


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    return average_price_rent['gross_rent'].hvplot(
        title='Average Gross Rent by Year',
        xlabel='Year',
        ylabel='Average Monthly Rent')
   

def average_sales_price():
    """Average Sales Price Per Year."""
    return average_price_rent['sale_price_sqr_foot'].hvplot(
        title='Average Price per Sqrft by Year',
        xlabel='Year',
        ylabel='Average price per Square Foot')
    


avg_price_neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    return avg_price_neighborhood.hvplot(x='year',
                       y='sale_price_sqr_foot', 
                       xlabel='Year',
                       ylabel='Average Sales Price per Square Foot', 
                       groupby='neighborhood')
    


avg_sales_neighborhood = avg_price_neighborhood.drop(columns=['year']).groupby('neighborhood').mean()
top10_expensive_neighborhood = avg_sales_neighborhood.sort_values("sale_price_sqr_foot", ascending = False).head(10)
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    return top10_expensive_neighborhood['sale_price_sqr_foot'].hvplot.bar(
        label='Top 10 Expensive Neighborhoods in SFO',
        xlabel='Neighborhood',
        shared_axes=False,
        ylabel='Avg. Sales Price per Square Foot', 
        rot=90)
    


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    return avg_price_neighborhood.drop(columns='housing_units').hvplot.bar(
        x='year',
        xlabel='Year',
        shared_axes=False,
        groupby='neighborhood',
        rot=90)

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    return px.parallel_coordinates(
        top10_expensive_neighborhood, 
        title='Parallel Coordinates Plot',
        color='sale_price_sqr_foot')
   


def parallel_categories():
    """Parallel Categories Plot."""
    
    return px.parallel_categories(
        top10_expensive_neighborhood.reset_index(), 
        color='sale_price_sqr_foot',
        title='Parallel Categories Plot')
        


mean_values_neighborhood=avg_sales_neighborhood.reset_index().rename(columns = {'neighborhood':'Neighborhood'})
neighborhood_locations= pd.merge(neighborhood_data, mean_values_neighborhood, on='Neighborhood')
def neighborhood_map():
    """Neighborhood Map."""

    return px.scatter_mapbox(neighborhood_locations,
                  lat="Lat",
                  lon="Lon",
                  size='sale_price_sqr_foot', 
                  color='gross_rent',
                  zoom=10,
                  hover_name='Neighborhood',
                  title='Average Sale Price per Square Foot and Gross Rent in San Francisco')




## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [78]:
# Create a Title for the Dashboard
dashboard_title = 'Real Estate Analysis of San Francisco from 2010 to 2016'


# Create a tab layout for the dashboard
welcome_tab = pn.Column(
    'This dashboard presents a visual analysis of historical prices of house units, sale prices per square foot and gross rent in San Francisco, California from 2010 to 2016. You can navigate through the tabs above to explore more details about the evolution of the real estate market on The Golden City across these years',
    neighborhood_map()
)
yearly_market_tab = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())
neighborhood_tab = pn.Column(average_price_by_neighborhood(),top_most_expensive_neighborhoods(),most_expensive_neighborhoods_rent_sales())

# Create the dashboard
SF_dashboard = pn.Tabs(
    ("Welcome", welcome_tab),
    ("Yearly Market Analysis", yearly_market_tab),
    ("Neighborhood", neighborhood_tab))

## Serve the Panel Dashboard

In [79]:
# Serve the# dashboard
SF_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()